In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [9]:
%load_ext autoreload
%autoreload 2

% cd /gdrive/My\ Drive/Colab\ Notebooks/nlp_task/
! pwd
! ls

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/gdrive/My Drive/Colab Notebooks/nlp_task
/gdrive/My Drive/Colab Notebooks/nlp_task
 bert			 classification.ipynb   nohup.out    simple_bert
 bert_cla		 data		        pointer      StarSpace
'bert for topic.ipynb'	 fine_gained	        pretrain
 Bert-Pretrain.ipynb	 multi_task-nlp-bert    pretrained


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.14.0

In [8]:
! export Bert_Model=./pretrained/chinese_L-12_H-768_A-12
! python ./bert/create_pretraining_data.py \
--input_file=./pretrain/bert_*_pretrain.txt \
--output_file=./pretrain/tf_pre_data.tfrecord \
--vocab_file=./pretrained/chinese_L-12_H-768_A-12/vocab.txt \
--do_lower_case=True \
--max_seq_length=256 \
--max_predictions_per_seq=60 \
--masked_lm_prob=0.15 \
--random_seed=12345 \
--dupe_factor=5

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [19]:
! python ./bert/run_pretraining.py \
--bert_config_file=./pretrained/chinese_L-12_H-768_A-12/bert_config.json \
--input_file=./pretrain/tf_pre_data.tfrecord \
--output_dir=./pretrained/fine_tune \
--init_checkpoint=./pretrained/chinese_L-12_H-768_A-12/bert_model.ckpt \
--learning_rate=2e-5 \
--do_train=True \
--do_eval=True \
--train_batch_size=16 \
--max_seq_length=256 \
--max_predictions_per_seq=60 \
--num_train_steps=5000 \
--num_warmup_steps=500 \
--masked_lm_prob=0.15 \
--random_seed=12345 \
--dupe_factor=5

流式输出内容被截断，只能显示最后 5000 行内容。
I0619 15:16:19.540015 139999200331648 tpu_estimator.py:2160] examples/sec: 8.88093
INFO:tensorflow:global_step/sec: 0.578853
I0619 15:16:21.266973 139999200331648 tpu_estimator.py:2159] global_step/sec: 0.578853
INFO:tensorflow:examples/sec: 9.26164
I0619 15:16:21.267348 139999200331648 tpu_estimator.py:2160] examples/sec: 9.26164
INFO:tensorflow:global_step/sec: 0.573394
I0619 15:16:23.010979 139999200331648 tpu_estimator.py:2159] global_step/sec: 0.573394
INFO:tensorflow:examples/sec: 9.1743
I0619 15:16:23.011243 139999200331648 tpu_estimator.py:2160] examples/sec: 9.1743
INFO:tensorflow:global_step/sec: 0.580288
I0619 15:16:24.734401 139999200331648 tpu_estimator.py:2159] global_step/sec: 0.580288
INFO:tensorflow:examples/sec: 9.28461
I0619 15:16:24.735022 139999200331648 tpu_estimator.py:2160] examples/sec: 9.28461
INFO:tensorflow:global_step/sec: 0.55863
I0619 15:16:26.524380 139999200331648 tpu_estimator.py:2159] global_step/sec: 0.55863
INFO:tensorflow

In [20]:
import re
import tensorflow as tf
from tensorflow.contrib.slim import get_variables_to_restore


# 将bert中和adam相关的参数的值去掉，较小模型的内存
graph = tf.Graph()
with graph.as_default():
    sess = tf.Session()
    checkpoint_file = tf.train.latest_checkpoint("./pretrained/fine_tune/")
    saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
    saver.restore(sess, checkpoint_file)

    variables = get_variables_to_restore()
    other_vars = [variable for variable in variables if not re.search("adam", variable.name)]
    var_saver = tf.train.Saver(other_vars)
    var_saver.save(sess, "./pretrained/fine_tune/model.ckpt")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./pretrained/fine_tune/model.ckpt-5000
